<a href="https://colab.research.google.com/github/hwankang/Pytorch_tutorials/blob/main/hwankang_0818_adadelta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline
import numpy as np
import pandas as pd

from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

# setosa_x = iris.data[:50]
# setosa_y = iris.target[:50]
# versicolor_x = iris.data[50:100]
# versicolor_y = iris.target[50:100]
# scatter(setosa_x[:, 0], setosa_x[:, 2])
# scatter(versicolor_x[:, 0], versicolor_x[:, 2])

# Extract sepal length, petal length from Setosa and Versicolor
data = iris.data[:100, [0, 2]]

# Standardization
scaler = StandardScaler()
data = scaler.fit_transform(data)


# Split data to test and train data
train_x, test_x, train_y, test_y = train_test_split(data, iris.target[:100], test_size=0.3)

# Plotting data
scatter(data[:50, 0], data[:50, 1], label='Setosa')
scatter(data[50:100, 0], data[50:100, 1], label='Versicolour')
title('Iris Data')
xlabel('sepal length')
ylabel('petal length')
grid()
legend()

In [ ]:
w =   np.array([ 0.09370901, -0.24480254, -0.84210235]) # np.random.randn(2 + 1)

def predict(w, x):
    N = len(x)
    yhat = w[1:].dot(x.T) + w[0]
    return yhat

def  adadelta_nn(w, X, Y, decay=0.2, epoch=4, weight_size=2):
    """
    @param eta <float>: learning rate
    """
    N = len(X)
    e = 1e-8

    Eg = np.zeros(weight_size + 1)  # E[g^2]
    Ed = np.zeros(weight_size + 1)  # E[\Delta w^2]

    i = 0
    for _ in range(epoch):
        for _ in range(N):
            x = X[i]
            y = Y[i]
            x = x.reshape((-1, 2))
            yhat = predict(w, x)

            # Calculate the gradients
            gradient_w = 2/N*-(y-yhat).dot(x)
            gradient_b = 2/N*-(y-yhat)

            # Accumulate Gradient
            Eg[1:] = decay * Eg[1:] + (1-decay) * gradient_w**2
            Eg[0]  = decay * Eg[0] + (1-decay) * gradient_b**2

            # Compute Update
            delta_w = - np.sqrt(e + Ed[1:])/np.sqrt(e + Eg[1:]) * gradient_w
            delta_b = - np.sqrt(e + Ed[0])/np.sqrt(e + Eg[0]) * gradient_b

            # Accumulate Updates
            Ed[1:] = decay * Ed[1:] + (1-decay) * delta_w**2
            Ed[0]  = decay * Ed[0]  + (1-decay) * delta_b**2

            w[1:] = w[1:] + delta_w
            w[0] = w[0] + delta_b

            i += 1
            if i >= N:
                i = 0

    return w


for _ in range(45):
    w = adadelta_nn(w, train_x, train_y)

    # Accuracy Test
    yhats = predict(w, test_x)
    yhats = np.where(yhats >= 0.5, 1, 0)
    accuracy = accuracy_score(test_y, yhats)
    print('Accuracy:', round(accuracy, 2), '\t\tweights:', w)

In [ ]:
w_w = 2/5*-(6-4)
w_w